In [ ]:
# default_exp __init__

# Core

> Team-strength models in Python

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import json
import datetime as dt
import numpy as np
import pprint

In [ ]:
#export

# For now, just re-export everything
from mezzala.adapters import *
from mezzala.blocks import *
from mezzala.models import *
from mezzala.weights import *
from mezzala.parameters import *

Let's demo

In [ ]:
with open('../data/premier-league-1516.json', 'r') as f:
    pl_1516 = json.load(f)

# Let's parse the dates, too
for match in pl_1516:
    match['date'] = dt.datetime.strptime(match['date'], '%Y-%m-%d')
    
pl_1516[0:3]

[{'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'Manchester United FC',
  'team2': 'Tottenham Hotspur FC',
  'score': {'ft': [1, 0]}},
 {'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'AFC Bournemouth',
  'team2': 'Aston Villa FC',
  'score': {'ft': [0, 1]}},
 {'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'Leicester City FC',
  'team2': 'Sunderland AFC',
  'score': {'ft': [4, 2]}}]

A model in `mezzala` is composed of 2 parts:

* Model blocks (see `mezzala.blocks`)
* An adapter (see `mezzala.adapters`)

The model blocks determine which terms your model estimates. In general, you will want
to estimate offensive and defensive strength for each team (`TeamStrength`) and 
as well as home advantage `HomeAdvantage`.

The selected model blocks can be supplied to the model as a list:

In [ ]:
blocks = [TeamStrength(), HomeAdvantage()]

An adapter connects your model to the data source. In other words, it tells the model how find the information needed to fit.

The information needed is determined by which model blocks are used. In our case,

* All models require `home_goals` and `away_goals`
* `TeamStrength` - requires `home_team` and `away_team`

`HomeAdvantage` doesn't require any information, since it assumes all matches have equal home-field advantage by default.

In [ ]:
adapter = KeyAdapter(               # `KeyAdapter` = data['...']
    home_team='team1',
    away_team='team2',
    home_goals=['score', 'ft', 0],  # Get nested fields with lists of fields
    away_goals=['score', 'ft', 1],  # i.e. data['score']['ft'][1]
)

Pulling this together, we can construct a model from an adapter and blocks

In [ ]:
%%prun
model = DixonColes(adapter=adapter, blocks=blocks)
model.fit(pl_1516)

# All estimates should be valid numbers
assert all(not np.isnan(x) for x in model.params.values())

# Home advantage should be positive
assert 1.0 < np.exp(model.params[HFA_KEY]) < 2.0

/Users/ben/code/mezzala/mezzala/models.py:134: RuntimeWarning: invalid value encountered in log
  np.log(self._tau(home_goals, away_goals, home_rate, away_rate, rho))


In [ ]:
param_keys = model.params.keys()
param_key_len = max(len(str(k)) for k in param_keys)

for k in param_keys:
    key_str = str(k).ljust(param_key_len + 1)
    print(f'{key_str}: {np.exp(model.params[k]):0.2f}')

Making predictions for a single match

In [ ]:
scorelines = model.predict_one({
    'team1': 'Manchester City FC',
    'team2': 'Swansea City FC',
})

scorelines[0:5]

In [ ]:
outcomes = scorelines_to_outcomes(scorelines)

list(outcomes.values())

Or for multiple matches

In [ ]:
many_scorelines = model.predict([
    {'team1': 'Manchester City FC',
     'team2': 'Swansea City FC'},
    {'team1': 'Manchester City FC',
     'team2': 'West Ham United FC'}
])

What about a model with a different weighting method?

By default, the `DixonColes` model weights all matches equally. However, it's more realistic to give matches
closer to the current date a bigger weight than those a long time ago.

The original Dixon-Coles paper suggests using an exponential weight, and we can use the same:

In [ ]:
season_end_date = max(match['date'] for match in pl_1516)

weight = ExponentialWeight(
    # Value of `epsilon` is taken from the original paper
    epsilon=-0.0065,  
    key=lambda x: (season_end_date - x['date']).days
)

In [ ]:
model_exp = DixonColes(
    adapter=adapter,
    blocks=blocks,
    weight=weight
)
model_exp.fit(pl_1516)

How much does that change the ratings at season-end?

In [ ]:
for k in param_keys:
    key_str = str(k).ljust(param_key_len + 1)
    model_param = np.exp(model.params[k])
    model_exp_param = np.exp(model_exp.params[k])
    print(f'{key_str}: {model_param:0.2f} -> {model_exp_param:0.2f} ({model_exp_param/model_param:0.2f})')